In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import matplotlib

In [8]:
import os
import logging
import argparse
import glob
import json
from collections import Counter

import numpy as np
from matplotlib.ticker import StrMethodFormatter

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from sklearn import metrics
import tensorflow as tf
import pandas as pd
from astropy.table import Table  # for NSA
from astropy import units as u
from sklearn.metrics import confusion_matrix, roc_curve
from PIL import Image
from scipy.stats import binom
from IPython.display import display, Markdown

from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

from shared_astro_utils import astropy_utils, matching_utils
from zoobot.estimators import make_predictions, bayesian_estimator_funcs
from zoobot.tfrecord import read_tfrecord
from zoobot.uncertainty import discrete_coverage
from zoobot.estimators import input_utils, losses
from zoobot.tfrecord import catalog_to_tfrecord
from zoobot.active_learning import metrics, simulated_metrics, acquisition_utils, check_uncertainty, simulation_timeline, run_estimator_config
from zoobot import label_metadata

In [9]:
from zoobot.active_learning.metrics import IterationState
import pickle

In [10]:
os.chdir('/home/walml/repos/zoobot')

In [11]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)


In [12]:

label_cols = label_metadata.gz2_partial_label_cols
questions = label_metadata.gz2_partial_questions

version = 'gz2'
schema = losses.Schema(label_cols, questions, version=version)



{smooth-or-featured, indices 0 to 1, asked after None: (0, 1)}


In [13]:
experiment_dir = 'results/temp/gz2_1q_1active_best'
state_loc = os.path.join(experiment_dir, 'iteration_0/state.pickle')

In [14]:

with open(state_loc, 'rb') as f:
    state = pickle.load(f)

In [15]:
def get_id_strs_in_shard(loc):
    feature_spec = input_utils.get_feature_spec({'id_str': 'string'})
    id_str_dataset = input_utils.get_dataset(loc, feature_spec, batch_size=1, shuffle=False, repeat=False, drop_remainder=False)
    id_strs = [str(d['id_str'].numpy().squeeze())[2:-1] for d in id_str_dataset]
    return id_strs

In [19]:
folder = '/home/walml/repos/zoobot/results/temp/gz2_1q_1active_best_it1_acquired'
locs = glob.glob(f'{folder}/acquired_tfrecords/*.tfrecord')
assert locs

In [24]:
ids_by_shard = [get_id_strs_in_shard(loc) for loc in locs]
ids_in_shards = [id_str for shard in ids_by_shard for id_str in shard]

In [26]:
len(ids_in_shards)

13653

In [31]:
len(set(ids_in_shards).intersection(set(state.id_strs)))

13653

In [40]:
np.any(np.isnan(state.acquisitions))

False

In [48]:
acquired_order = np.argsort(state.acquisitions)[::-1]
acquired_id_strs = [state.id_strs[i] for i in acquired_order][:len(ids_in_shards)]

In [51]:
ids_in_shards[0], acquired_id_strs[0]  # the actual order of acquisition is random...

('dr7objid_588848900473028789', 'dr7objid_587733431925080181')

In [49]:
len(set(ids_in_shards).intersection(set(acquired_id_strs)))  # ...but they should all be acquired

13653

Okay, the right ids are being acquired. How about the right images and labels?